In [2]:
using Pkg
dev_folder = "../Examples/"
pkg_folder = "../"
Pkg.activate(dev_folder)
Pkg.develop(path=pkg_folder)

  Activating project at `~/BindingAndCatalysis.jl/Examples`
   Resolving package versions...
     Project No packages added to or removed from `~/BindingAndCatalysis.jl/Examples/Project.toml`
    Manifest No packages added to or removed from `~/BindingAndCatalysis.jl/Examples/Manifest.toml`


In [3]:
using BindingAndCatalysis
using Revise
using GLMakie

In [5]:
Pkg.precompile("BindingAndCatalysis")

In [6]:
using BindingAndCatalysis

[ Info: Precompiling BindingAndCatalysis [b532a4b1-2c45-4c12-bcaf-8695372aa40c] (cache misses: wrong dep version loaded (4))

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to print a failed Task notice; giving up

SYSTEM: caught exception of type :MethodError while trying to prin

LoadError: Failed to precompile BindingAndCatalysis [b532a4b1-2c45-4c12-bcaf-8695372aa40c] to "/home/joker/.julia/compiled/v1.12/BindingAndCatalysis/jl_QcV3kJ".

In [2]:
N = [1 0 0 0 1 -1 0 0 0 0; 0 1 0 0 1 0 -1 0 0 0; 0 0 0 0 0 1 1 -1 0 0; 1 0 0 0 0 0 0 1 -1 0; 0 0 0 0 0 0 1 0 1 -1]

5×10 Matrix{Int64}:
 1  0  0  0  1  -1   0   0   0   0
 0  1  0  0  1   0  -1   0   0   0
 0  0  0  0  0   1   1  -1   0   0
 1  0  0  0  0   0   0   1  -1   0
 0  0  0  0  0   0   1   0   1  -1

In [3]:
# N = N_generator(5,10)
model = Bnc(N=N)

----------Binding Network Summary:-------------
Number of species (n): 10
Number of conserved quantities (d): 5
Number of reactions (r): 5
L matrix: [1 0 … 2 2; 0 1 … 1 2; … ; 0 0 … 0 0; 0 0 … 2 3]
N matrix: [1 0 … 0 0; 0 1 … 0 0; … ; 1 0 … -1 0; 0 0 … 1 -1]
Direction of binding reactions: backward
Catalysis involved: No
Regimes constructed: No
-----------------------------------------------

In [ ]:
find_all_vertices!(model)
summary(model)

In [ ]:
get_H(model,1)

In [ ]:
get_perm(model,10)

In [ ]:
get_nullity(model,20)

In [ ]:
poly = get_polyhedron(model,13)

In [ ]:
get_nullity(poly)

In [ ]:
get_one_inner_point(model,15)

In [ ]:
vtx = get_vertex(model,16)

In [ ]:
vtxs = get_vertices(model,singular=false,asymptotic=true)[6:10]

In [ ]:
Cs = vtxs .|> x->get_C(model,x)
C0s = vtxs .|> x->get_C0(model,x)

In [4]:
calc_volume(model)

---------------------Start finding all vertices, it may takes a while.--------------------
Finished, with 79 vertices found and 51 asymptotic vertices.

-------------Start calculating nullity for each vertex, it also takes a while.------------
1.Building Nρ_inv cache in parallel...
2.Calculating nullity for each vertex in parallel...
3.Storing all vertices information...
Done.


[ Info: Number of polyhedra to calc volume: 29
[ Info: Reached time limit (40.02 s). Stopping.


79-element Vector{Tuple{Float64, Float64}}:
 (0.11729372512532947, 0.0005757067030601153)
 (0.03742869423715569, 0.00018600963562926914)
 (0.005449711736461464, 2.7220349745283604e-5)
 (0.019329211086800246, 9.662022917477019e-5)
 (0.0022005342450889453, 1.5435811852841868e-5)
 (0.043793456617250044, 0.00021751421093128584)
 (0.08179867027954826, 0.00040036156344256)
 (0.04611804486526253, 0.00022980230514511952)
 (0.018523518240497295, 9.228719653665339e-5)
 (0.003649291150022844, 1.9863474346173986e-5)
 (0.014969692472936584, 7.452085936248693e-5)
 (0.055965656053710476, 0.0002793924046686629)
 (0.0, 0.0)
 ⋮
 (0.01621136792064979, 8.07315896860035e-5)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)
 (0.0, 0.0)

In [ ]:
BindingAndCatalysis.calc_volume_old(Cs, C0s)

In [ ]:
Cs = Matrix.(Cs)

In [ ]:
calc_volume(Cs, C0s)

In [ ]:
vtx_grh = BindingAndCatalysis.get_vertices_graph!(model,full=false)

In [ ]:
@show Threads.nthreads() Threads.maxthreadid() Threads.threadid()